In [1]:
import tensorflow as tf
import numpy as np
import random
import time
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
import msvcrt
import sys




In [2]:

tf.reset_default_graph()

dataset_path      = "dataset10/"

test_labels_file  = "TestLabel.txt"
train_labels_file = "TrainLabel.txt"
valid_labels_file = "ValidLabel.txt"

logs_path			= "logs/"

NUM_DIM=100
NUM_LABELS		= 21
SEED					= 123
TRAIN_BATCH_SIZE	= 0
VALID_BATCH_SIZE	= 2100
TEST_BATCH_SIZE		= 2100
TRAIN_MIN_QUEUE_SIZE=  TRAIN_BATCH_SIZE
TEST_MIN_QUEUE_SIZE=  TEST_BATCH_SIZE 
TOTRAL_TRAIN_SIZE=0

def encode_label(label):
    return int(label)

def read_label_file(file):
    f = open(file, "r")
    filepaths = []
    labels = []
    for line in f:
        filepath, label = line.split(" ")
        filepaths.append(filepath)
        labels.append(encode_label(label))
    return filepaths, labels




# reading labels and file path
train_filepaths, train_labels = read_label_file(dataset_path + train_labels_file)
valid_filepaths, valid_labels = read_label_file(dataset_path + valid_labels_file)
test_filepaths, test_labels = read_label_file(dataset_path + test_labels_file)



# transform relative path into full path
train_filepaths = [ dataset_path+"Train/" + fp for fp in train_filepaths]
valid_filepaths = [ dataset_path+"Valid/" + fp for fp in valid_filepaths]
test_filepaths = [ dataset_path+"Test/" + fp for fp in test_filepaths]
TOTAL_TRAIN_SIZE=len(train_filepaths)
TRAIN_BATCH_SIZE=(int)(TOTAL_TRAIN_SIZE*0.1)


# convert string into tensors
train_contents = ops.convert_to_tensor(train_filepaths, dtype=dtypes.string)
train_labels = tf.one_hot(train_labels,depth=NUM_LABELS,on_value=1,off_value=0,axis=-1)

valid_contents = ops.convert_to_tensor(valid_filepaths, dtype=dtypes.string)
valid_labels = tf.one_hot(valid_labels,depth=NUM_LABELS,on_value=1,off_value=0,axis=-1)

test_contents = ops.convert_to_tensor(test_filepaths, dtype=dtypes.string)
test_labels = tf.one_hot(test_labels,depth=NUM_LABELS,on_value=1,off_value=0,axis=-1)




# create input queues
train_input_queue = tf.train.slice_input_producer(
																		[train_contents, train_labels],
																		shuffle=True)

# create input queues
valid_input_queue = tf.train.slice_input_producer(
																		[valid_contents, valid_labels],
																		shuffle=True)

test_input_queue = tf.train.slice_input_producer(
																		[test_contents, test_labels],
																		shuffle=True)

record_defaults=[]
for i in range(NUM_DIM):
    record_defaults.append(tf.constant([1], dtype=tf.float32))



# process path and string tensor into an image and a label
file_content = tf.read_file(train_input_queue[0])
train_content=  tf.decode_csv(file_content, record_defaults=record_defaults)
train_label = train_input_queue[1]

file_content = tf.read_file(valid_input_queue[0])
valid_content=  tf.decode_csv(file_content, record_defaults=record_defaults)
valid_label = valid_input_queue[1]


file_content = tf.read_file(test_input_queue[0])
test_content=  tf.decode_csv(file_content, record_defaults=record_defaults)  
test_label = test_input_queue[1]

train_content= tf.stack(train_content)
valid_content= tf.stack(valid_content)
test_content= tf.stack(test_content)

# define tensor shape
train_content.set_shape([NUM_DIM])
valid_content.set_shape([NUM_DIM])
test_content.set_shape([NUM_DIM])


# collect batches of images before processing
train_content_batch, train_label_batch =  tf.train.batch(
																		[train_content, train_label],
																		batch_size=TRAIN_BATCH_SIZE
 																		)
valid_content_batch, valid_label_batch =  tf.train.batch(
																		[valid_content, valid_label],
																		batch_size=VALID_BATCH_SIZE
																		)
test_content_batch, test_label_batch =  tf.train.batch(
																		[test_content, test_label],
																		batch_size=TEST_BATCH_SIZE
																		)

print ("input pipeline ready")

input pipeline ready


In [3]:
def selu(x, name):
    with ops.name_scope('elu') as scope:
        alpha = 1.6732632423543772848170429916717
        scale = 1.0507009873554804934193349852946
        return scale*tf.where(x>=0.0, x, alpha*tf.nn.elu(x,name))


# placeholder is used for feeding data.
x = tf.placeholder(tf.float32, shape=[None, NUM_DIM], name = 'x') # none represents variable length of dimension. 784 is the dimension of MNIST data.
y_target = tf.placeholder(tf.float32, shape=[None, NUM_LABELS], name = 'y_target') # shape argument is optional, but this is useful to debug.


# reshape input data
x_data = tf.reshape(x, [-1,NUM_DIM], name="x_data")

# Build a fully connected layer
W_fc1 = tf.Variable(tf.truncated_normal([NUM_DIM, 1024],stddev=np.sqrt(1/NUM_DIM)), name = 'W_fc1')
b_fc1 = tf.Variable(tf.random_normal([1024],stddev=0), name = 'b_fc1')
h_fc1 = selu(tf.matmul(x_data, W_fc1) + b_fc1, name="h_fc1")

shape = h_fc1.get_shape().as_list()
flat_num= shape[1]

# Build a fully connected layer
W_fc2 = tf.Variable(tf.truncated_normal([flat_num, 512], stddev=np.sqrt(1/flat_num)), name = 'W_fc2')
b_fc2 = tf.Variable(tf.random_normal([512],stddev=0), name = 'b_fc2')
h_fc2 = selu(tf.matmul(h_fc1, W_fc2) + b_fc2, name="h_fc2")

shape = h_fc2.get_shape().as_list()
flat_num= shape[1]

# Build a fully connected layer
W_fc3 = tf.Variable(tf.truncated_normal([flat_num, 256], stddev=np.sqrt(1/flat_num)), name = 'W_fc3')
b_fc3 = tf.Variable(tf.random_normal([256],stddev=0), name = 'b_fc3')
h_fc3 = selu(tf.matmul(h_fc2, W_fc3) + b_fc3, name="h_fc3")

shape = h_fc3.get_shape().as_list()
flat_num= shape[1]

# Build a fully connected layer
W_fc4 = tf.Variable(tf.truncated_normal([flat_num, 128], stddev=np.sqrt(1/flat_num)), name = 'W_fc4')
b_fc4 = tf.Variable(tf.random_normal([128],stddev=0), name = 'b_fc4')
h_fc4 = selu(tf.matmul(h_fc3, W_fc4) + b_fc4, name="h_fc4")

shape = h_fc4.get_shape().as_list()
flat_num= shape[1]


W_fc5 = tf.Variable(tf.random_normal([flat_num, 64], stddev=np.sqrt(1/flat_num)), name = 'W_fc5')
b_fc5 = tf.Variable(tf.random_normal([64],stddev=0), name = 'b_fc5')
h_fc5 =selu(tf.matmul(h_fc4, W_fc5) + b_fc5, name="h_fc5")

shape = h_fc5.get_shape().as_list()
flat_num= shape[1]

W_fc6 = tf.Variable(tf.random_normal([flat_num, NUM_LABELS], stddev=np.sqrt(1/flat_num)), name = 'W_fc6')
b_fc6 = tf.Variable(tf.random_normal([NUM_LABELS],stddev=0), name = 'b_fc6')


pred=tf.matmul(h_fc5, W_fc6) + b_fc6

prob_y=tf.nn.softmax(pred, name="prob_y")

# define the Loss function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y_target))
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# define optimization algorithm
train_step = tf.train.AdamOptimizer(1e-4).minimize(cost)


correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y_target, 1))
# correct_prediction is list of boolean which is the result of comparing(model prediction , data)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 

#save weight
t_vars = tf.trainable_variables()
saver = tf.train.Saver(max_to_keep=None,var_list=t_vars)
saver_def = saver.as_saver_def()

In [4]:
#config = tf.ConfigProto(log_device_placement=True)
#config.gpu_options.per_process_gpu_memory_fraction=0.3 # don't hog all vRAM
#config.operation_timeout_in_ms=60000   # terminate on long hangs
configure=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True))

sess = tf.Session(config=configure) 

training_loss=tf.summary.scalar('training_loss', cost)
validation_loss=tf.summary.scalar('validation_loss', cost)

training_accuracy = tf.summary.scalar("training_accuracy", accuracy)
validation_accuracy = tf.summary.scalar("validation_accuracy", accuracy)


# Merge all summaries into a single op
merged_op = tf.summary.merge_all()

writer=tf.summary.FileWriter(logs_path, sess.graph)

# initialization
#init_op = tf.global_variables_initializer()
init_op = tf.global_variables_initializer()

In [5]:
print("Session started!")
start_session_time = time.time()
sess.run(init_op)

# initialize the queue threads to start to shovel data
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess,coord=coord)

max_epoch=70
display_step=1
max_accuracy=0
max_index=0
for epoch in range(0,max_epoch+1):
    avg_cost = 0
    total_batch=(int)(TOTAL_TRAIN_SIZE/TRAIN_BATCH_SIZE)
    for step in range(total_batch):
        train_batch=sess.run([tf.cast(train_content_batch, tf.float32),tf.cast(train_label_batch, tf.float32)])
        if epoch!=0:
            _,c=sess.run([train_step,cost] , feed_dict={x: train_batch[0], y_target: train_batch[1]})
            # Compute average loss
            avg_cost += c / total_batch
        else:
            c=sess.run(cost , feed_dict={x: train_batch[0], y_target: train_batch[1]})
            avg_cost += c / total_batch

    # Display logs per epoch step
    if epoch % display_step == 0:

        print ("Epoch:", '%04d' % (epoch), "cost=","{:.9f}".format(avg_cost))

        valid_batch=sess.run([tf.cast(valid_content_batch, tf.float32),tf.cast(valid_label_batch, tf.float32)])

        # traininig accuracy.
        train_cos,train_acc, train_summ_acc,train_summ_loss = sess.run(
        [cost, accuracy, training_accuracy,training_loss], 
        feed_dict={x : train_batch[0],  y_target : train_batch[1]})
        writer.add_summary(train_summ_acc, epoch) 
        writer.add_summary(train_summ_loss, epoch) 


        # validation accuracy.
        valid_cos, valid_acc, valid_summ_acc,valid_summ_loss  = sess.run(
        [cost,accuracy, validation_accuracy,validation_loss],
        feed_dict={x: valid_batch[0], y_target: valid_batch[1]})
        writer.add_summary(valid_summ_acc, epoch)
        writer.add_summary(valid_summ_loss, epoch)

        saver.save(sess,"./weight/w",epoch)

        print("Train-Accuracy:", train_acc,"Train-Loss:", train_cos, "Validation-Accuracy:", valid_acc,"Val-Loss:", valid_cos,"\n")

        if valid_acc>max_accuracy:
            max_accuracy=valid_acc
            max_index=epoch



saver.restore(sess,"./weight/w-%d"%(max_index))
test_batch=sess.run([tf.cast(test_content_batch,tf.float32),tf.cast(test_label_batch, tf.float32)])
max_accuracy=sess.run(accuracy, feed_dict={x: test_batch[0], y_target:  test_batch[1]})

print("(result)test accuracy: %g / weight-%d"%(max_accuracy,max_index))    

# stop our queue threads and properly close the session
coord.request_stop()
coord.join(threads)
writer.close()
sess.close()

print("close")

Session started!
Epoch: 0000 cost= 3.234940386
INFO:tensorflow:./weight/w-0 is not in all_model_checkpoint_paths. Manually adding it.
Train-Accuracy: 0.0920635 Train-Loss: 3.24278 Validation-Accuracy: 0.0709524 Val-Loss: 3.23323 

Epoch: 0001 cost= 2.442936254
INFO:tensorflow:./weight/w-1 is not in all_model_checkpoint_paths. Manually adding it.
Train-Accuracy: 0.685714 Train-Loss: 1.75428 Validation-Accuracy: 0.7 Val-Loss: 1.73488 

Epoch: 0002 cost= 1.409700727
INFO:tensorflow:./weight/w-2 is not in all_model_checkpoint_paths. Manually adding it.
Train-Accuracy: 0.892063 Train-Loss: 1.08928 Validation-Accuracy: 0.897143 Val-Loss: 1.10173 

Epoch: 0003 cost= 0.932139683
INFO:tensorflow:./weight/w-3 is not in all_model_checkpoint_paths. Manually adding it.
Train-Accuracy: 0.914286 Train-Loss: 0.820767 Validation-Accuracy: 0.940476 Val-Loss: 0.764253 

Epoch: 0004 cost= 0.655564302
INFO:tensorflow:./weight/w-4 is not in all_model_checkpoint_paths. Manually adding it.
Train-Accuracy: 0.9